In [ ]:
import pandas as pd 
import datetime
from pandas import to_datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.dates as mdates
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
import mitosheet as sheet

## Import and Format Data

In [ ]:
df = pd.read_csv('Modelling Data.csv')#read modelling data

In [ ]:
df.head() #inspect top 5 rows of dataset

In [ ]:
df.tail() #inspect bottom 5 rows of dataset

In [ ]:
df.info() #get information on dataset

In [ ]:
df['Q_num'] = (df.Quarter.astype(str)).str[5] 
df.head()

In [ ]:
df['Quarter'] = pd.to_datetime(df['Quarter'])#format Quarter as datetime instead of object. This is needed in order to plot dataset as a time series.
df.info() #get info on reformatted dataset

In [ ]:
df.head()#inspect top 5 rows of reformatted dataframe

In [ ]:
df = df.set_index('Quarter') #set 'Quarter' as the dataframe index

In [ ]:
df.head() #inspect top 5 rows of reformatted dataframe

In [ ]:
test_years = [4,3,2] #define three test year periods: 4 years, 3 years and 2 years

## SARIMA

### Run model for each set of training data and each train/test period. Plot results

In [ ]:
# Import the model we are using
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
model_name = "SARIMA" #define model name
#write for loop to iterate over the 3 train/test periods
for i in test_years:
    training_data = df[['VALUE']].iloc[:-i*4] #set the training data equal to df_SV less the last 16 (4*4), 12 (3*4) and 8 (2*4) quarters
    testing_data = df[['VALUE']].tail(i*4) #set testing data equal to the last 16 (4*4), 12 (3*4) and 8 (2*4) quarters of df_SV
    y=training_data
    SARIMAXmodel = SARIMAX(y, order = (1,1,1), seasonal_order=(1,1,1,12)) #'12' used as the seasonality in the time series is repeated every calendar year
    SARIMAXmodel = SARIMAXmodel.fit()
    predictions = SARIMAXmodel.get_forecast(len(testing_data.index))
    predictions = predictions.conf_int(alpha = 0.05) 
    predictions["Predictions"] = SARIMAXmodel.predict(start = predictions.index[0],
                                            end = predictions.index[-1])
    predictions.index = testing_data.index
    y_pred = pd.DataFrame(predictions["Predictions"]) #'predictions' is an array - convert to a dataframe
    exec(f'{model_name}_Predictions_{i}y = y_pred')
    exec(f'{model_name}_MAE_{i}y = mean_absolute_error(testing_data.values, y_pred["Predictions"])')#calculate and save MAE of the model predictions for each train/test period
    exec(f'{model_name}_RMSE_{i}y= np.sqrt(mean_squared_error(testing_data.values, y_pred["Predictions"]))')#calculate and save RMSE of the model predictions for each train/test period

## Random Forest Regressor

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df_array = np.array(df.Q_num) #convert Q_num to an array
df_array = df_array.reshape(-1, 1) #reshape array so that it can be processed by the Random Forest model

In [ ]:
df_array #inspect the array

### Run model for each set of training data and each train/test period. Plot results

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model_name = "RF" #define model name
#write for loop to iterate over the 3 training/testing periods
for i in test_years:
    train_features = df_array[:-i*4]
    test_features = df_array[-i*4:len(df_array)]
    labels = np.array(df['VALUE'])
    train_labels = labels[:-i*4]
    test_labels =  labels[-i*4:len(df_array)]
    # Instantiate model with 1000 decision trees
    rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
    # Train the model on training data
    rf.fit(train_features, train_labels);
    # Use the forest's predict method on the test data
    predictions = rf.predict(test_features)  
    training_data = df[['VALUE']].iloc[:-i*4] #for plotting only, set the training data equal to the full df_SV less the last 16 (4*4), 12 (3*4) and 8 (2*4) quarters
    testing_data = df[['VALUE']].tail(i*4) #for plotting only, set testing data equal to the last 16 (4*4), 12 (3*4) and 8 (2*4) quarters of df_SV
    y_pred = pd.DataFrame(predictions, columns=['Predictions']) #'predictions' is an array - convert to a dataframe
    y_pred.index = testing_data.index
    exec(f'{model_name}_Predictions_{i}y = y_pred')
    exec(f'{model_name}_MAE_{i}y = mean_absolute_error(testing_data.values, y_pred["Predictions"])')#calculate and save MAE of the model predictions for each train/test period
    exec(f'{model_name}_RMSE_{i}y= np.sqrt(mean_squared_error(testing_data.values, y_pred["Predictions"]))')#calculate and save RMSE of the model predictions for each train/test period 

## LSTM

### Format data for LSTM model

In [ ]:
array = df[['VALUE']].iloc[:, 0].values #Transform dataframe to an array as the keras module of TensorFlow only accepts NumPy arrays as parameters 

### Run model for each set of training data and each train/test period. Plot results

In [ ]:
#Import classes from TensorFlow that are needed to build the RNN 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
scaler = MinMaxScaler() #define MinMaxScaler() as 'scaler'

#### Specify 1 Timestep

In [ ]:
model_name = "LSTM_1ts" #define model name
timesteps  = 1  #specify 1 timestep - this means that the model uses the last timestep to predict the next timestep
#write for loop to iterate over the 3 training/testing periods
for i in test_years: 
    training_data = array[:-i*4] #set the training data as an array equal to the full Seasonal Variation dataset less the last 16 (4*4), 12 (3*4) and 8 (2*4) quarters
    testing_data_ = array[-i*4:len(array)] #set testing data as an array equal to the last 16 (4*4), 12 (3*4) and 8 (2*4) quarters
    scaled_training_data = scaler.fit_transform(training_data.reshape(-1, 1)) #Scale testing data so that it fits between -1 and +1
    scaled_testing_data_ = scaler.fit_transform(testing_data_.reshape(-1, 1)) #Scale testing data so that it fits between -1 and +1
    x_training_data = [] #Create empty list called x_training_data
    y_training_data =[] #Create empty list called y_training data
    for j in range(timesteps, len(scaled_training_data)): #Specify 4 timesteps. For every quarter's seasonal variation the model predicts, it will use 4 previous quarters pricing to determine its output.  
        x_training_data.append(scaled_training_data[j-timesteps:j, 0]) #Use for loop to populate actual data into the  and y_training_data lists
        y_training_data.append(scaled_training_data[j, 0])
    x_training_data = np.array(x_training_data) #TensorFlow is designed to work with arrays so we need to transform the two Python lists we created in the previous step into NumPy arrays.
    y_training_data = np.array(y_training_data)
    x_training_data = np.reshape(x_training_data, (x_training_data.shape[0], #Reshape the training data one more time because the RNN layer available in TensorFlow only accepts data in a very specific format.  
    x_training_data.shape[1],1))
    
    rnn = Sequential() #Initialise an object from TensorFlows Sequential class. This class is designed to build neural networks by adding sequences of layers over time.
    rnn.add(LSTM(units = 50, return_sequences = True, input_shape =(x_training_data.shape[1], 1))) #Add an LSTM layer first. Specify 3 arguments: Units = 50, return_sequences = True, input_shape = 4 timesteps and  1 predictor 
    rnn.add(Dropout(0.2)) #Dropout regularisation is a technique used to avoid overfitting when training neural networks. Assume a dropout rate of 20%.
    rnn.add(LSTM(units = 50, return_sequences = True)) #Add three more LSTM layers
    rnn.add(Dropout(0.2))
    rnn.add(LSTM(units = 50, return_sequences = True))
    rnn.add(Dropout(0.2))
    rnn.add(LSTM(units = 50)) #Remove return_sequences = True as this is the last LSTM layer
    rnn.add(Dropout(0.2))
    rnn.add(Dense(units = 1)) #Add output layer. Units = 1 as we want to predict the next quarter's seasonal variation
    rnn.compile(optimizer = 'adam', loss = 'mean_absolute_error') #Specify Adam optimiser. Since we are predicting a continuous variable we can use Mean Squared Error as our loss parameter.
    rnn.fit(x_training_data, y_training_data, epochs = 100, batch_size = 32)  #Train the RNN on our training data ( and y_training_data). Epochs = number of iterations I want the RNN to train on. Batch size = size of batches RNN will be trained on through each epoch.
    
    x_test_data = df[['VALUE']][len(df[['VALUE']])-i*4-timesteps:] #we need 4 quarters (testing data) set + 1 timestep =  5 quarters
    x_test_data = x_test_data.iloc[:,0].values
    x_test_data.shape #x_test_data should have 5 rows and no columns
    x_test_data = np.reshape(x_test_data, (-1, 1)) #reshape NumPy array to make it suitable for the predict method
    x_test_data = scaler.transform(x_test_data) #Scale test data so we can use model to make predictions. This is because we want to transform the test data according to the fit generated from
#the entire training data set. This means that the transformation that is applied to the test data will be the same as the one applied to the training data - which is necessary for our recurrent neural network to make
#accurate predictions
    final_x_test_data = []                             
    for j in range(timesteps, len(x_test_data)):
        final_x_test_data.append(x_test_data[j-timesteps:j, 0])
    final_x_test_data = np.array(final_x_test_data)
    final_x_test_data = np.reshape(final_x_test_data, (final_x_test_data.shape[0],final_x_test_data.shape[1],1)) #reshape final test data to meet TensorFlow standards
    predictions = rnn.predict(final_x_test_data) #Pass final_x_test_data into the predict() method called on the run object and store in a variable called predictions. 
    unscaled_predictions = scaler.inverse_transform(predictions) #unscale predictions data
    training_data = df[['VALUE']].iloc[:-i*4] #for plotting only, set the training data equal to the full df_SV less the last 16 (4*4), 12 (3*4) and 8 (2*4) quarters
    testing_data = df[['VALUE']].tail(i*4) #for plotting only, set testing data equal to the last 16 (4*4), 12 (3*4) and 8 (2*4) quarters of df_SV
    y_pred = pd.DataFrame(unscaled_predictions, columns=['Predictions']) #'unscaled_predictions' is an array - convert to a dataframe
    y_pred.index = testing_data.index
    exec(f'{model_name}_Predictions_{i}y = y_pred')
    exec(f'{model_name}_MAE_{i}y = mean_absolute_error(testing_data.values, y_pred["Predictions"])')#calculate and save MAE of the model predictions for each train/test period
    exec(f'{model_name}_RMSE_{i}y= np.sqrt(mean_squared_error(testing_data.values, y_pred["Predictions"]))')#calculate and save RMSE of the model predictions for each train/test period

#### Specify 4 Timesteps

In [ ]:
model_name = "LSTM_4ts" #define model name
timesteps  = 4 #specify 1 timestep - this means that the model uses the last 4 timesteps to predict the next 4 timesteps
#write for loop to iterate over the 3 training/testing periods
for i in test_years: 
    training_data = array[:-i*4] #set the training data as an array equal to the full Seasonal Variation dataset less the last 16 (4*4), 12 (3*4) and 8 (2*4) quarters
    testing_data_ = array[-i*4:len(array)] #set testing data as an array equal to the last 16 (4*4), 12 (3*4) and 8 (2*4) quarters
    scaled_training_data = scaler.fit_transform(training_data.reshape(-1, 1)) #Scale testing data so that it fits between -1 and +1
    scaled_testing_data_ = scaler.fit_transform(testing_data_.reshape(-1, 1)) #Scale testing data so that it fits between -1 and +1
    x_training_data = [] #Create empty list called x_training_data
    y_training_data =[] #Create empty list called y_training data
    for j in range(timesteps, len(scaled_training_data)): #Specify 4 timesteps. For every quarter's seasonal variation the model predicts, it will use 4 previous quarters pricing to determine its output.  
        x_training_data.append(scaled_training_data[j-timesteps:j, 0]) #Use for loop to populate actual data into the  and y_training_data lists
        y_training_data.append(scaled_training_data[j, 0])
    x_training_data = np.array(x_training_data) #TensorFlow is designed to work with arrays so we need to transform the two Python lists we created in the previous step into NumPy arrays.
    y_training_data = np.array(y_training_data)
    x_training_data = np.reshape(x_training_data, (x_training_data.shape[0], #Reshape the training data one more time because the RNN layer available in TensorFlow only accepts data in a very specific format.  
    x_training_data.shape[1],1))
    
    rnn = Sequential() #Initialise an object from TensorFlows Sequential class. This class is designed to build neural networks by adding sequences of layers over time.
    rnn.add(LSTM(units = 50, return_sequences = True, input_shape =(x_training_data.shape[1], 1))) #Add an LSTM layer first. Specify 3 arguments: Units = 50, return_sequences = True, input_shape = 4 timesteps and  1 predictor 
    rnn.add(Dropout(0.2)) #Dropout regularisation is a technique used to avoid overfitting when training neural networks. Assume a dropout rate of 20%.
    rnn.add(LSTM(units = 50, return_sequences = True)) #Add three more LSTM layers
    rnn.add(Dropout(0.2))
    rnn.add(LSTM(units = 50, return_sequences = True))
    rnn.add(Dropout(0.2))
    rnn.add(LSTM(units = 50)) #Remove return_sequences = True as this is the last LSTM layer
    rnn.add(Dropout(0.2))
    rnn.add(Dense(units = 1)) #Add output layer. Units = 1 as we want to predict the next quarter's seasonal variation
    rnn.compile(optimizer = 'adam', loss = 'mean_absolute_error') #Specify Adam optimiser. Since we are predicting a continuous variable we can use Mean Absolute Error as our loss parameter.
    rnn.fit(x_training_data, y_training_data, epochs = 100, batch_size = 32)  #Train the RNN on our training data ( and y_training_data). Epochs = number of iterations I want the RNN to train on. Batch size = size of batches RNN will be trained on through each epoch.
    
    x_test_data = df[['VALUE']][len(df[['VALUE']])-i*4-timesteps:] #we need 4 quarters (testing data) set + 1 timestep =  5 quarters
    x_test_data = x_test_data.iloc[:,0].values
    x_test_data.shape #x_test_data should have 5 rows and no columns
    x_test_data = np.reshape(x_test_data, (-1, 1)) #reshape NumPy array to make it suitable for the predict method
    x_test_data = scaler.transform(x_test_data) #Scale test data so we can use model to make predictions. This is because we want to transform the test data according to the fit generated from
#the entire training data set. This means that the transformation that is applied to the test data will be the same as the one applied to the training data - which is necessary for our recurrent neural network to make
#accurate predictions
    final_x_test_data = []                             
    for j in range(timesteps, len(x_test_data)):
        final_x_test_data.append(x_test_data[j-timesteps:j, 0])
    final_x_test_data = np.array(final_x_test_data)
    final_x_test_data = np.reshape(final_x_test_data, (final_x_test_data.shape[0],final_x_test_data.shape[1],1)) #reshape final test data to meet TensorFlow standards
    predictions = rnn.predict(final_x_test_data) #Pass final_x_test_data into the predict() method called on the run object and store in a variable called predictions. 
    unscaled_predictions = scaler.inverse_transform(predictions) #unscale predictions data
    training_data = df[['VALUE']].iloc[:-i*4] #for plotting only, set the training data equal to the full df_SV less the last 16 (4*4), 12 (3*4) and 8 (2*4) quarters
    testing_data = df[['VALUE']].tail(i*4) #for plotting only, set testing data equal to the last 16 (4*4), 12 (3*4) and 8 (2*4) quarters of df_SV
    y_pred = pd.DataFrame(unscaled_predictions, columns=['Predictions']) #'unscaled_predictions' is an array - convert to a dataframe
    y_pred.index = testing_data.index
    exec(f'{model_name}_Predictions_{i}y = y_pred')
    exec(f'{model_name}_MAE_{i}y = mean_absolute_error(testing_data.values, y_pred["Predictions"])')#calculate and save MAE of the model predictions for each train/test period
    exec(f'{model_name}_RMSE_{i}y= np.sqrt(mean_squared_error(testing_data.values, y_pred["Predictions"]))')#calculate and save RMSE of the model predictions for each train/test period

## Prophet

### Format data for Prophet model

In [ ]:
df_Prophet = df[['VALUE']] #create new dataframe called df_Prophet 
df_Prophet.reset_index(inplace=True) #reset the index
df_Prophet.columns = ['ds','y'] #rename the columns to 'ds' and 'y' in order for the Prophet model to process the input data
df_Prophet['ds'] = to_datetime(df_Prophet['ds']) #convert the 'ds' column to datetime format 

In [ ]:
df_Prophet.head() #inspect the first 5 rows of the dataframe

### Run model for each set of training data and each train/test period. Plot results

In [ ]:
# Import the model we are using
from prophet import Prophet

In [ ]:
model_name = "Prophet" #define model name
#run for loop to iterate over the 3 different test year periods
for i in test_years: 
    df_train = df_Prophet[:-i*4]
    model = Prophet()
    model.fit(df_train)
    future = df_Prophet[-i*4:len(df_Prophet)]
    future.columns = ['ds','y']
    forecast = model.predict(future)
    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])  # summarise the forecast
    model.plot(forecast) # plot forecast
    y_pred = forecast[['ds','yhat']]
    y_pred =  y_pred.set_index('ds')
    training_data = df_train.set_index('ds') #for plotting, set the 'ds' column of the training data as the index
    testing_data_ = df_Prophet[-i*4:len(df_Prophet)] #for plotting only, set testing data equal to the last 16 (4*4), 12 (3*4) and 8 (2*4) quarters
    testing_data_ = testing_data_.set_index('ds') #for plotting, set the 'ds' column of the testing data to the index
    y_pred = y_pred.rename(columns={'yhat': 'Predictions'})
    exec(f'{model_name}_Predictions_{i}y = y_pred')
    exec(f'{model_name}_MAE_{i}y = mean_absolute_error(testing_data_.values, y_pred["Predictions"])') #calculate and save MAE of the model predictions for each train/test period
    exec(f'{model_name}_RMSE_{i}y= np.sqrt(mean_squared_error(testing_data_.values, y_pred["Predictions"]))') #calculate and save RMSE of the model predictions for each train/test period

## Model Evaluation

### Rank Model Results by MAE & RMSE

In [ ]:
df_MAE = pd.DataFrame() #create an empty DataFrame
# append columns to an empty DataFrame
df_MAE['Model'] = ['SARIMA','Random Forest','LSTM - 1 Timestep', 'LSTM - 4 Timesteps','Prophet'] #define column names
df_MAE['MAE (4 Year Testing Period)'] = [SARIMA_MAE_4y,RF_MAE_4y,LSTM_1ts_MAE_4y,LSTM_4ts_MAE_4y,Prophet_MAE_4y] #populate 'MAE 4 year' column with MAE_4y result for each model
df_MAE['MAE (3 Year Testing Period)'] = [SARIMA_MAE_3y,RF_MAE_3y,LSTM_1ts_MAE_3y,LSTM_4ts_MAE_3y,Prophet_MAE_3y] #populate 'MAE 3 year' column with MAE_4y result for each model
df_MAE['MAE (2 Year Testing Period)'] = [SARIMA_MAE_2y,RF_MAE_2y,LSTM_1ts_MAE_2y,LSTM_4ts_MAE_2y,Prophet_MAE_2y] #populate 'MAE 2 year' column with MAE_4y result for each model
df_MAE['MAE (Average)'] = (df_MAE['MAE (4 Year Testing Period)'] + df_MAE['MAE (3 Year Testing Period)'] + df_MAE['MAE (2 Year Testing Period)'])/3 #populate 'MAE Average'column with average MAE across the 3 training periods for each model
df_MAE = df_MAE.set_index('Model') #set 'Model' column equal to the dataframe index
df_MAE.sort_values(by='MAE (Average)', ascending=True) #sort MAE results in ascending order

In [ ]:
df_RMSE = pd.DataFrame() #create an empty DataFrame
# append columns to an empty DataFrame
df_RMSE['Model'] = ['SARIMA','Random Forest', 'LSTM - 1 Timestep','LSTM - 4 Timesteps','Prophet'] #define column names
df_RMSE['RMSE (4 Year Testing Period)'] = [SARIMA_RMSE_4y,RF_RMSE_4y,LSTM_1ts_RMSE_4y,LSTM_4ts_RMSE_4y,Prophet_RMSE_4y] #populate 'RMSE 4 year' column with RMSE_4y result for each model
df_RMSE['RMSE (3 Year Testing Period)'] = [SARIMA_RMSE_3y,RF_RMSE_3y,LSTM_1ts_RMSE_3y,LSTM_4ts_RMSE_3y,Prophet_RMSE_3y] #populate 'RMSE 3 year' column with RMSE_3y result for each model
df_RMSE['RMSE (2 Year Testing Period)'] = [SARIMA_RMSE_2y,RF_RMSE_2y,LSTM_1ts_RMSE_2y,LSTM_4ts_RMSE_2y,Prophet_RMSE_2y] #populate 'RMSE 2 year' column with RMSE_3y result for each model
df_RMSE['RMSE (Average)'] = (df_RMSE['RMSE (4 Year Testing Period)'] + df_RMSE['RMSE (3 Year Testing Period)'] + df_RMSE['RMSE (2 Year Testing Period)'])/3 #populate 'RMSE Average'column with average RMSE across the 3 training periods for each model
df_RMSE = df_RMSE.set_index('Model') #set 'Model' column equal to the dataframe index
df_RMSE.sort_values(by='RMSE (Average)', ascending=True) #sort RMSE results in ascending order

In [ ]:
#collate model predictions for 4 year test period
All_Predictions_4y = df[['VALUE']].tail(16)
All_Predictions_4y.rename(columns= {'VALUE':'Actual'},inplace=True)
All_Predictions_4y['SARIMA'] = SARIMA_Predictions_4y.Predictions
All_Predictions_4y['Random Forest'] = RF_Predictions_4y.Predictions
All_Predictions_4y['LSTM (1 Timestep)'] = LSTM_1ts_Predictions_4y.Predictions
All_Predictions_4y['LSTM (4 Timesteps)'] = LSTM_4ts_Predictions_4y.Predictions
All_Predictions_4y['Prophet'] = Prophet_Predictions_4y.Predictions
All_Predictions_4y = All_Predictions_4y.reset_index()

#collate model predictions for 3 year test period
All_Predictions_3y = df[['VALUE']].tail(12)
All_Predictions_3y.rename(columns= {'VALUE':'Actual'},inplace=True)
All_Predictions_3y['SARIMA'] = SARIMA_Predictions_3y.Predictions
All_Predictions_3y['Random Forest'] = RF_Predictions_3y.Predictions
All_Predictions_3y['LSTM (1 Timestep)'] = LSTM_1ts_Predictions_3y.Predictions
All_Predictions_3y['LSTM (4 Timesteps)'] = LSTM_4ts_Predictions_3y.Predictions
All_Predictions_3y['Prophet'] = Prophet_Predictions_3y.Predictions
All_Predictions_3y = All_Predictions_3y.reset_index()

#collate model predictions for 2 year test period
All_Predictions_2y = df[['VALUE']].tail(8)
All_Predictions_2y.rename(columns= {'VALUE':'Actual'},inplace=True)
All_Predictions_2y['SARIMA'] = SARIMA_Predictions_2y.Predictions
All_Predictions_2y['Random Forest'] = RF_Predictions_2y.Predictions
All_Predictions_2y['LSTM (1 Timestep)'] = LSTM_1ts_Predictions_2y.Predictions
All_Predictions_2y['LSTM (4 Timesteps)'] = LSTM_4ts_Predictions_2y.Predictions
All_Predictions_2y['Prophet'] = Prophet_Predictions_2y.Predictions
All_Predictions_2y = All_Predictions_2y.reset_index()

In [ ]:
from mitosheet import sheet
sheet = sheet(All_Predictions_4y, analysis_to_replay="id-ojagjlwgkf")